In [1]:
from hdf5storage import loadmat, savemat 
import numpy as np 

from scipy import signal 
from scipy.fftpack import fft

from sklearn.covariance import GraphicalLassoCV

import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__)) 
# make sure it is newer version, works in version 1.2.1

The scikit-learn version is 1.2.1.


In [2]:
# load abs_zscore_all
outdict=loadmat('abs_zscore_all.mat')
abs_zscore_all=outdict["abs_zscore_all"]

In [3]:
# append abs_zscore in all trials together
abs_zscore_all_append = list()
for ses in range(12):
    for subj in range(2):
        for trl in range(12):
            abs_zscore_all_append.append(abs_zscore_all[ses][trl][subj])

ampdata=np.concatenate(abs_zscore_all_append)

In [4]:
def partialcorrelation(ampdata,ncv=5):
    pc_lasso = GraphicalLassoCV(cv=ncv)
    nbin = np.shape(ampdata)[1]
    nchan = np.shape(ampdata)[2]
    pcorr = np.zeros((nbin,nchan,nchan))
    alphas = np.zeros(nbin)
    # likelihoods = list()
    likelihoods = np.zeros(nbin)
    covmat = np.zeros((nbin,nchan,nchan))
    for f in range(nbin):
        xx= np.abs(np.squeeze(ampdata[:,f,:]))
        pc_lasso.fit(xx) # the system is too ill for this solver
        covariance = pc_lasso.covariance_
        precision = pc_lasso.precision_
        alphas[f] = pc_lasso.alpha_
        dict_cvresults=pc_lasso.cv_results_
        best_alpha_ind=np.where(dict_cvresults['alphas']==pc_lasso.alpha_)
        # likelihoods.append(pc_lasso.cv_results_["mean_test_score"])
        likelihoods[f] = pc_lasso.cv_results_["mean_test_score"][best_alpha_ind][0]
        parcor = np.zeros((32,32))
        y = np.diag(precision)
        yy = np.outer(y,y)
        yy = np.sqrt(yy)
        parcor = precision/yy
        pcorr[f,:,:] = parcor
        y = np.diag(covariance)
        yy = np.outer(y,y)
        yy = np.sqrt(yy)
        parcor_cov = covariance/yy        
        covmat[f,:,:] = parcor_cov 
    return pcorr,covmat,alphas,likelihoods  


In [5]:
# parameters 
epoch = 1 #units: seconds
maxf = 30 #units: Hz
maxbin = maxf*epoch #convert maxf into number of bins to save

In [6]:
# fit lasso model for each of the 4 conditions in 2 syn types
pcorr,covmat,alphas,likelihoods = partialcorrelation(ampdata,ncv=5)

outdict = dict()
outdict['pcorr'] = pcorr
outdict['covmat']= covmat, 
outdict['alpha'] = alphas
outdict['likelihood'] = likelihoods

# take 0.5 min

/home/zhibinz2/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/zhibinz2/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/zhibinz2/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [13]:
# save syn_pcorr

savemat('pcorr_single',outdict,store_python_metadata=True)

